# 정적 웹 크롤링과 스크래핑을 이용한 공공데이터와 SNS 데이터 수집

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req
import io

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
savename = "C:/Temp/forecast.xml"
req.urlretrieve(url, savename)

xml = open(savename, "r", encoding="utf-8").read()
soup = BeautifulSoup(xml, 'xml') # xml

info = {}
for location in soup.find_all("location"):
    loc = location.find('city').string
    min_w = location.find_all('tmn')
    max_w = location.find_all('tmx')
    weather = [a.string+"~"+b.string for a, b in zip(min_w, max_w)]

    if not (loc in info):
        info[loc] = []
    for data in weather:
        info[loc].append(data)
print(info)

with open('C:/Temp/forecast.txt', "wt", encoding="utf-8") as f:
    for loc in sorted(info.keys()):
        f.write(str(loc)+'\n')
        for name in info[loc]:
            f.write('\t'+str(name)+'\n')

{'서울': ['7~15', '7~15', '5~14', '5~14', '6~13', '6~13', '5~12', '5~12', '4~12', '4~12', '4~11', '2~9', '3~10'], '인천': ['8~14', '8~14', '6~13', '6~13', '8~13', '8~13', '7~12', '7~12', '6~12', '6~12', '5~11', '4~9', '5~10'], '수원': ['6~16', '6~16', '4~14', '4~14', '6~14', '6~14', '4~13', '4~13', '3~13', '3~13', '4~12', '1~10', '2~11'], '파주': ['3~14', '3~14', '1~13', '1~13', '3~13', '3~13', '1~12', '1~12', '2~12', '2~12', '2~11', '-1~9', '0~10'], '이천': ['3~15', '3~15', '2~14', '2~14', '4~14', '4~14', '2~13', '2~13', '1~12', '1~12', '1~12', '0~10', '1~11'], '평택': ['5~17', '5~17', '4~16', '4~16', '7~16', '7~16', '4~14', '4~14', '4~14', '4~14', '3~13', '1~10', '2~11'], '춘천': ['4~15', '4~15', '3~13', '3~13', '5~13', '5~13', '3~12', '3~12', '3~12', '3~12', '2~11', '0~9', '-2~8'], '원주': ['5~15', '5~15', '4~14', '4~14', '6~13', '6~13', '3~12', '3~12', '2~12', '2~12', '2~12', '1~11', '-1~8'], '강릉': ['11~18', '11~18', '9~14', '9~14', '10~15', '10~15', '8~15', '8~15', '7~16', '7~16', '7~15', '6~14',

In [2]:
import datetime
from bs4 import BeautifulSoup
import urllib.request

now = datetime.datetime.now()
nowDate = now.strftime('%Y년 %m월 %d일 %H시 %M분 입니다.')
print(nowDate)
# 오늘의 날씨
print('[ 오늘의 날씨 요약 ]')
webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('서울날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 서울 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('광주날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 광주 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('제주도날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 제주도 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

2022년 11월 18일 09시 08분 입니다.
[ 오늘의 날씨 요약 ]
--> 서울 날씨 : 6.5℃ 맑음
--> 광주 날씨 : 6.7℃ 맑음
--> 제주도 날씨 : 14.0℃ 구름많음


In [3]:
from bs4 import BeautifulSoup
import urllib.request as req

busNum = '360'
key = '%2BjzsSyNtwmcqxUsGnflvs3rW2oceFvhHR8AFkM3ao%2Fw50hwHXgGyPVutXw04uAXvrkoWgkoScvvhlH7jgD4%2FRQ%3D%3D'
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum
res = req.urlopen(url1)

soup = BeautifulSoup(res.read().decode('utf-8'), "xml")

busRouteId = None
for itemList in soup.find_all('itemList') :
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey='+key+'&busRouteId='+busRouteId
res = req.urlopen(url2)
soup = BeautifulSoup(res.read(), "xml")

busPos = []
for itemList in soup.find_all('itemList') :
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string

    busPos.append((gpsY, gpsX))

print('[ ' + busNum + '번 버스의 운행 위치 ]')
if len(busPos) != 0 :
    print(busNum + '번 버스 ' + str(len(busPos)) + '대 운행중...')
    for lat,lng in busPos :
        print(lat+','+lng)
else :
    print('현재 운행중인 ' + busNum + '번 버스가 없어요...')

print(url1)
print(url2)

[ 360번 버스의 운행 위치 ]
360번 버스 91대 운행중...
37.4699297672,127.1275897549
37.4790433896,127.1261237834
37.482125,127.124748
37.4883648963,127.1213627915
37.4943822012,127.1171609467
37.5002084826,127.1118384337
37.5034082968,127.1089442853
37.5068001738,127.1058262006
37.5128993133,127.0979457604
37.5122132454,127.0939144028
37.5117722886,127.0872805398
37.5118706155,127.0833904794
37.512009235,127.0796916235
37.5111175337,127.0720732937
37.5099462683,127.0657996068
37.5086670057,127.0617005963
37.5066831874,127.0554269279
37.505143,127.050205
37.503232,127.044057
37.5015409767,127.0386328674
37.5004418394,127.0350786317
37.4988294041,127.0298390986
37.500785,127.02637
37.506367,127.023728
37.5106480618,127.0192203041
37.5083418804,127.0121292217
37.5062923789,127.0052834138
37.5034152049,126.9958263651
37.5019451598,126.9900264523
37.503188597,126.9766525988
37.505867,126.971748
37.5064370015,126.9685844969
37.5075850724,126.9655921789
37.5115728985,126.9612375315
37.512734203,126.9509621786

In [5]:
from bs4 import BeautifulSoup
import urllib.request as req

key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '100'
url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/LampScpgmtb/'+startIndex+'/'+endIndex+'/'
res = req.urlopen(url)
xml = res.read()
soup = BeautifulSoup(xml, "xml")

pjList = []
for itemList in soup.find_all('row') :
    up_nm = itemList.find('UP_NM').string
    up_nm = '없음' if up_nm is None else up_nm
    pgm_nm = itemList.find('PGM_NM').string
    pgm_nm = '없음' if pgm_nm is None else pgm_nm
    target_nm = itemList.find('TARGET_NM').string
    target_nm = '없음' if target_nm is None else target_nm
    u_price = itemList.find('U_PRICE').string
    u_price = '없음' if u_price is None else u_price
    pjList.append((up_nm, pgm_nm, target_nm, u_price))

print('[ 서울 청소년 수련관 강좌 리스트 ]')
for up_nm,pgm_nm,target_nm,u_price in pjList :
    print(up_nm+','+pgm_nm+','+target_nm+','+str(u_price))

[ 서울 청소년 수련관 강좌 리스트 ]
금천청소년수련관,아침B-청소년,주2회수영-청소년(구),36000
금천청소년수련관,난초A,초4~24세미만,41000
금천청소년수련관,아침C(5회)-청소년,주5회수영-청소년(구),56000
시립서대문청소년센터,경기민요와 장구 A,7세~초등,40000
시립서대문청소년센터,국가공인 한자지도사,성인,30000
시립서대문청소년센터,기린A,초2~5,30000
시립서대문청소년센터,상어1-3(초급),초등학생,40000
시립서대문청소년센터,셀프리더쉽&표현력 스피치 A,초3~6,33000
시립서대문청소년센터,속성 제빵자격증반,초6~성인(3개월),150000
시립서대문청소년센터,신나는미술세상(C),5~6세,30000
시립서대문청소년센터,신나는미술세상(D),초등학생,30000
시립서대문청소년센터,죠스2-1(상급),초등학생,33000
시립서대문청소년센터,죠스2-1(상급3개월) ,초등학생,89100
시립서대문청소년센터,죠스2-2(중급),초등학생,33000
광진청소년센터,목요바이올린(7세-성인)B,성인,61000
마포청소년수련관,재미있는통기타교실(토12시40분),초3-성인,35000
동대문청소년수련관,청소년성장집단상담,구분없음,1000000
시립서대문청소년센터,방학)체스(A),7세~초등,35000
시립서대문청소년센터,방학)체스(B),7세~초등,35000
시립서대문청소년센터,실버 (3개월),성인,청소년(14세부터),142560
금천청소년수련관,난초A중급-청소년,주3회수영-청소년,41000
금천청소년수련관,매화C(5회),초4~24세미만,56000
금천청소년수련관,물개유아A,만5세~초3,36000
창동청소년센터,9/7 1일특강 취미난타(성인),성인,10000
창동청소년센터,특)우드아트,청소년,10000
노원청소년센터,21학년-(수업료)-7세 사자,체르니30(청소년),610000
화곡청소년센터,미라 소세지 빵10/16(토)A-10시30분(초1-초6),초등,6000
중구청소년센터,[12월]주말농구대관-최대욱,누구나(화목),110000
잠실청소년센터,우쿨렐레 

In [6]:
from bs4 import BeautifulSoup
import urllib.request as req
import io

key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '1000'
date = '20220815'

url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/CardSubwayStatsNew/'+startIndex+'/'+endIndex+'/'+date+'/'
print(url)
res = req.urlopen(url)
xml = res.read()

soup = BeautifulSoup(xml, "xml")

subwayList = []
for itemList in soup.find_all('row') :
    line_num = itemList.find('LINE_NUM').string
    sub_sta_nm = itemList.find('SUB_STA_NM').string
    ride_pasgr_num = itemList.find('RIDE_PASGR_NUM').string
    alight_pasgr_num = itemList.find('ALIGHT_PASGR_NUM').string
    subwayList.append((line_num, sub_sta_nm, ride_pasgr_num, alight_pasgr_num))

print('[ 서울시 지하철호선별 역별 승하차 인원 정보 ]')
for line_num, sub_sta_nm, ride_pasgr_num, alight_pasgr_num in subwayList :
    print(line_num+','+sub_sta_nm+','+ride_pasgr_num+','+alight_pasgr_num)


http://openapi.seoul.go.kr:8088/796143536a756e69313134667752417a/xml/CardSubwayStatsNew/1/1000/20220815/
[ 서울시 지하철호선별 역별 승하차 인원 정보 ]
1호선,서울역,32920,29633
1호선,시청,19918,19136
1호선,종각,18931,16095
1호선,종로3가,17198,15589
1호선,종로5가,15078,14474
1호선,동대문,7840,8053
1호선,신설동,6500,6685
1호선,제기동,11834,12488
1호선,청량리(서울시립대입구),13636,14305
1호선,동묘앞,7302,7898
2호선,시청,9773,8006
2호선,을지로입구,19197,17122
2호선,을지로3가,7609,7045
2호선,을지로4가,5759,5503
2호선,동대문역사문화공원(DDP),8399,9729
2호선,신당,7897,8640
2호선,상왕십리,6928,6860
2호선,왕십리(성동구청),8406,7573
2호선,한양대,2352,2570
2호선,뚝섬,8467,8963
2호선,성수,14426,15747
2호선,건대입구,23763,25528
2호선,구의(광진구청),11216,11496
2호선,강변(동서울터미널),24540,19874
2호선,잠실나루,5077,5292
2호선,잠실(송파구청),47082,44389
2호선,잠실새내,11206,11594
2호선,종합운동장,3114,3546
2호선,삼성(무역센터),20135,19626
2호선,선릉,13186,12109
2호선,역삼,10417,11623
2호선,강남,30953,29257
2호선,교대(법원.검찰청),10237,11448
2호선,서초,5576,4987
2호선,방배,5755,6201
2호선,사당,19169,20982
2호선,낙성대(강감찬),13387,14181
2호선,서울대입구(관악구청),23642,24949
2호선,봉천,11625,11829
2호선,신림,32467,32644
2호선,신대방,13935,14083
2호선,구로디지털단지

In [ ]:
import urllib.request
import json
client_key = 'izGsqP2exeThwwEUVU3x'
client_secret = 'WrwbQ1l6ZI'
query = '스마트폰'
encText = urllib.parse.quote_plus(query)
num = 100
naver_url = 'https://openapi.naver.com/v1/search/blog.json?query=' + encText + '&display=' + str(num)

request = urllib.request.Request(naver_url)
request.add_header("X-Naver-Client-Id",client_key)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    dataList = json.loads(response_body)

    print('[' + query + '에 대한 네이버 블로그 글 ]')
    for count, data in enumerate(dataList['items'], 1) :
        print (str(count) + ' : ' + data['title'])
        print ('[' + data['description'] + ']')
else:
    print('오류 코드 : ' + rescode)

In [ ]:
dataList # JSON

In [ ]:
dataList["items"][0]["description"]

In [ ]:
title = []
desc = []
for data in dataList["items"]:
    title.append(data["title"])
    desc.append(data["description"])  

In [ ]:
len(title)

In [ ]:
len(desc)

In [ ]:
title

In [ ]:
import re
for i in range(len(title)) :
   title[i] = re.sub("</?b>", "", title[i])

In [ ]:
for i in range(len(desc)) :
   desc[i] = re.sub("</?b>", "", desc[i])

In [ ]:
import pandas as pd
dictdata = {"title":title,"description":desc}
df = pd.DataFrame(dictdata)
df

In [ ]:
df.to_csv("output/smartphone.csv")

In [ ]:
import urllib.request
import json

client_key = 'izGsqP2exeThwwEUVU3x'
client_secret = 'WrwbQ1l6ZI'
query = '오징어게임'
encText = urllib.parse.quote_plus(query)

num = 100
naver_url = 'https://openapi.naver.com/v1/search/news.json?query=' + encText + '&display=' + str(num)

request = urllib.request.Request(naver_url)
request.add_header("X-Naver-Client-Id",client_key)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)

rescode = response.getcode()

if(rescode == 200):
    response_body = response.read()
    dataList = json.loads(response_body)
    count = 1
    print('[' + query + '에 대한 네이버 뉴스 글 ]')
    for data in dataList['items'] :
        print (str(count) + ' : ' + data['title'])
        print ('[' + data['description'] + ']')
        count += 1
else:
    print('오류 코드 : ' + rescode)
